# LSTM Autoencoder — Training & Evaluation (FD001)

Baseline reconstruction model:
- stacked LSTM encoder–decoder (128→64→latent=32 → decoder mirrored),
- loss: MSE, variance-normalized scoring,
- validation percentile sweep for threshold (F1-optimal),
- warm-up calibration on test (K=50), run-length filter=3,
- saves metrics, scores, and PR curve points to `artifacts/`.


## 1) Load Config & Set Seeds
- `config.yaml` -> `training_lstm`, `evaluation`
- deterministic TF ops (if available)


In [1]:
import os, json, yaml, hashlib, time, random, shutil, sys, subprocess
from pathlib import Path
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, models, callbacks, optimizers
from sklearn.metrics import precision_recall_fscore_support, average_precision_score, f1_score, precision_recall_curve
import matplotlib.pyplot as plt

with open("config.yaml", "r", encoding="utf-8") as f:
    CFG = yaml.safe_load(f)

T = CFG["training_lstm"]
E = CFG["evaluation"]


In [2]:
def set_seeds(seed=42, tf_deterministic=True):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed); np.random.seed(seed); tf.random.set_seed(seed)
    if tf_deterministic:
        os.environ["TF_DETERMINISTIC_OPS"] = "1"
        try: tf.config.experimental.enable_op_determinism(True)
        except: pass
set_seeds(CFG["repro"]["seed"], CFG["repro"].get("tf_deterministic", True))

# Postproc helpers
def run_length_filter(pred_bin, min_run=1):
    if min_run <= 1: return pred_bin.astype(int)
    y = pred_bin.astype(int).copy()
    i = 0
    while i < len(y):
        if y[i] == 1:
            j = i
            while j < len(y) and y[j] == 1: j += 1
            if (j - i) < min_run: y[i:j] = 0
            i = j
        else: i += 1
    return y

def summarize_metrics(y_true, y_pred, scores):
    prec, rec, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="binary", zero_division=0
    )
    pr_auc = average_precision_score(y_true, scores)
    return {"precision": float(prec), "recall": float(rec),
            "f1": float(f1), "pr_auc": float(pr_auc)}



## 2) Load Arrays & Sanity Checks
 - Uses preprocessed arrays saved by the preprocessing pipeline
 - Confirms `config.yaml` hash matches preprocessing (or warns)


In [3]:
Xtr = np.load("artifacts/X_train_normal.npy")
Xva = np.load("artifacts/X_val.npy")
yva = np.load("artifacts/y_val.npy").astype(int)
META = json.load(open("artifacts/preprocessing_meta.json","r",encoding="utf-8"))

L, D = Xtr.shape[1], Xtr.shape[2]
print(f"[shapes] Xtr={Xtr.shape}  Xva={Xva.shape}  yva={yva.shape}")

# Sanity: YAML drift guard
def yaml_md5(path="config.yaml"):
    with open(path, "rb") as f: return hashlib.md5(f.read()).hexdigest()
if META["yaml_hash"] != yaml_md5("config.yaml"):
    print("WARNING: YAML drift detected (config.yaml changed since preprocessing).")
    print("Proceeding because seq_len/stride/scaling must match.")



[shapes] Xtr=(7487, 80, 15)  Xva=(2524, 80, 15)  yva=(2524,)


## 3) Build Model — LSTM Autoencoder

**Architecture**
- Stacked LSTM encoder–decoder:
  - Encoder: LSTM(enc1) → LSTM(enc2) → LSTM(latent)
  - Decoder: RepeatVector(seq_len) → LSTM(enc2) → LSTM(enc1) → TimeDistributed(Dense(n_features))
- Dropout applied after recurrent layers.
- Hyperparameters read from `config.yaml` → `training_lstm` (enc1, enc2, latent, dropout, loss, lr).

**Inputs & shapes**
- Sequence length = `seq_len` from preprocessing (e.g., 80)
- Features = number of selected sensors after preprocessing

**What this cell does**
- Builds the Keras model for the current shapes
- Compiles with MSE  in YAML
- Prints a model summary for reproducibility


In [4]:
def build_lstm_ae(seq_len, n_feats, enc1=128, enc2=64, latent=32, dropout=0.2):
    In = layers.Input(shape=(seq_len, n_feats))
    x = layers.LSTM(enc1, return_sequences=True)(In); x = layers.Dropout(dropout)(x)
    x = layers.LSTM(enc2, return_sequences=True)(x); x = layers.Dropout(dropout)(x)
    z = layers.LSTM(latent)(x); z = layers.Dropout(dropout)(z)
    y = layers.RepeatVector(seq_len)(z)
    y = layers.LSTM(enc2, return_sequences=True)(y); y = layers.Dropout(dropout)(y)
    y = layers.LSTM(enc1, return_sequences=True)(y); y = layers.Dropout(dropout)(y)
    Out = layers.TimeDistributed(layers.Dense(n_feats))(y)
    return models.Model(In, Out, name="lstm_ae")

model = build_lstm_ae(L, D, T["enc1"], T["enc2"], T["latent"], T.get("dropout",0.2))
loss = "mse" if T.get("loss","mse").lower()=="mse" else keras.losses.Huber(delta=float(T.get("huber_delta",1.0)))
opt = keras.optimizers.Adam(learning_rate=T["lr"])
model.compile(optimizer=opt, loss=loss)
model.summary()

Model: "lstm_ae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 80, 15)]          0         
                                                                 
 lstm (LSTM)                 (None, 80, 128)           73728     
                                                                 
 dropout (Dropout)           (None, 80, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 80, 64)            49408     
                                                                 
 dropout_1 (Dropout)         (None, 80, 64)            0         
                                                                 
 lstm_2 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_2 (Dropout)         (None, 32)                0   

## 4) Train — Callbacks & Checkpoints
 - Train on normal windows only
- Validation = normal windows from X_val (if present), else `validation_split`
 - Callbacks: EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
 - Saves: `lstm_ae_best.weights.h5`, `lstm_ae_final.weights.h5`


In [5]:
ART = "artifacts"; os.makedirs(ART, exist_ok=True)
ckpt_best = os.path.join(ART, "lstm_ae_best.weights.h5")


Xva_norm = Xva[yva == 0]
use_val_split = len(Xva_norm) == 0  

es  = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=int(T.get("early_stopping_patience", 10)),
    min_delta=float(T.get("early_stopping_min_delta", 1e-4)),
    restore_best_weights=True,
    verbose=1,
)
rlr = callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=int(T.get("rlr_patience", 5)),
    min_delta=float(T.get("early_stopping_min_delta", 1e-4)),
    cooldown=2,
    min_lr=1e-6,
    verbose=1,
)
mc  = callbacks.ModelCheckpoint(
    ckpt_best, monitor="val_loss", save_best_only=True, save_weights_only=True, verbose=1
)

if use_val_split:
    print("[train] No normal windows in X_val → using validation_split from Xtr.")
    hist = model.fit(
        Xtr, Xtr,
        validation_split=T.get("val_frac", 0.10),
        epochs=T["epochs"],
        batch_size=T["batch"],
        shuffle=True,
        callbacks=[es, rlr, mc],
        verbose=1,
    )
else:
    print(f"[train] Using X_val normals for validation: n={len(Xva_norm)}")
    hist = model.fit(
        Xtr, Xtr,
        validation_data=(Xva_norm, Xva_norm),
        epochs=T["epochs"],
        batch_size=T["batch"],
        shuffle=True,
        callbacks=[es, rlr, mc],
        verbose=1,
    )

model.save_weights(os.path.join(ART, "lstm_ae_final.weights.h5"))


[train] Using X_val normals for validation: n=1844
Epoch 1/100
117/117 [==============================] - ETA: 0s - loss: 0.5316
Epoch 1: val_loss improved from inf to 0.45591, saving model to artifacts\lstm_ae_best.weights.h5
117/117 [==============================] - 17s 56ms/step - loss: 0.5316 - val_loss: 0.4559 - lr: 5.0000e-04
Epoch 2/100
116/117 [============================>.] - ETA: 0s - loss: 0.4564
Epoch 2: val_loss improved from 0.45591 to 0.43686, saving model to artifacts\lstm_ae_best.weights.h5
117/117 [==============================] - 5s 46ms/step - loss: 0.4562 - val_loss: 0.4369 - lr: 5.0000e-04
Epoch 3/100
116/117 [============================>.] - ETA: 0s - loss: 0.4417
Epoch 3: val_loss improved from 0.43686 to 0.42628, saving model to artifacts\lstm_ae_best.weights.h5
117/117 [==============================] - 5s 46ms/step - loss: 0.4416 - val_loss: 0.4263 - lr: 5.0000e-04
Epoch 4/100
116/117 [============================>.] - ETA: 0s - loss: 0.4353
Epoch 4: val_

## 3) Validation Scoring & Threshold Sweep (LSTM-AE)

- Compute variance-normalized mean squared error (per-feature variance from training residuals).  
- Sweep thresholds across percentiles (70–99.9) to maximize F1 on the validation set.  
- Save outputs:
  - `feat_var_lstm.npy` — variance vector for normalization  
  - `scores_lstm_val.npy` — anomaly scores for validation set  
  - `sweep_lstm_val.npy` — sweep results (p, threshold, precision, recall, F1)  
  - `metrics_lstm_val.json` — summary of chosen threshold and metrics  
  - `pr_lstm_val.npy` — precision–recall curve data


In [ ]:
pred_tr = model.predict(Xtr, verbose=0)
res_tr  = (pred_tr - Xtr)**2
feat_var = np.clip(res_tr.reshape(-1, D).var(axis=0), 1e-8, None)

pred_va = model.predict(Xva, verbose=0)
res_va  = (pred_va - Xva)**2
scores_va = (res_va / feat_var[None, None, :]).mean(axis=(1, 2))


pmin = float(E.get("sweep", {}).get("pmin", 70.0))
pmax = float(E.get("sweep", {}).get("pmax", 99.9))
step = float(E.get("sweep", {}).get("step", 0.1))

cands = np.arange(pmin, pmax + 1e-9, step, dtype=float)

best = {"f1": -1.0}
sweep_rows = []
for p in cands:
    thr = np.percentile(scores_va, p)
    yhat = (scores_va >= thr).astype(int)
    yhat = run_length_filter(yhat, int(E.get("min_run_length", 1)))
    prec, rec, f1, _ = precision_recall_fscore_support(yva, yhat, average="binary", zero_division=0)
    pr_auc = average_precision_score(yva, scores_va)
    sweep_rows.append([p, float(thr), float(prec), float(rec), float(f1)])
    if f1 > best["f1"]:
        best = {"p": float(p), "thr": float(thr), "f1": float(f1), "yhat": yhat}

m_val = summarize_metrics(yva, best["yhat"], scores_va)
print("[VAL]", m_val, f"thr={best['thr']:.6f} pct={best['p']:.2f}")


np.save(os.path.join(ART, "feat_var_lstm.npy"), feat_var)
np.save(os.path.join(ART, "scores_lstm_val.npy"), scores_va)
np.save(os.path.join(ART, "sweep_lstm_val.npy"), np.array(sweep_rows, dtype=float))  # columns: p,thr,prec,rec,f1

val_payload = {
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "metrics": m_val,
    "threshold_mode": "val_percentile_sweep_varnorm",
    "val_percentile": best["p"],
    "threshold_value": best["thr"],
    "postproc": {"min_run_length": int(E.get("min_run_length", 1))},
    "scoring": {"type": "variance_normalized_mean"},
    "sweep": {"pmin": pmin, "pmax": pmax, "step": step},
}
with open(os.path.join(ART, "metrics_lstm_val.json"), "w", encoding="utf-8") as f:
    json.dump(val_payload, f, indent=2)


prec_va, rec_va, thr_va = precision_recall_curve(yva, scores_va)
np.save(os.path.join(ART, "pr_lstm_val.npy"), np.c_[rec_va[:-1], prec_va[:-1], thr_va])

print("Saved: feat_var_lstm.npy, scores_lstm_val.npy, sweep_lstm_val.npy, metrics_lstm_val.json, pr_lstm_val.npy")


[VAL] {'precision': 0.8282097649186256, 'recall': 0.6735294117647059, 'f1': 0.7429034874290349, 'pr_auc': 0.8375025879916973} thr=2.081047 pct=78.10
Saved: feat_var_lstm.npy, scores_lstm_val.npy, sweep_lstm_val.npy, metrics_lstm_val.json, pr_lstm_val.npy


## 6) Test Scoring, Warm-Up Calibration, PR Curves
 - Compute raw test scores
 - Warm-up z-score per engine (first K=50 windows), then fixed threshold from validation
 - Saves: `scores_lstm_test.npy`, `scores_lstm_test_varnorm_warmup.npy`,
          `pr_lstm_test.npy`, `pr_lstm_test_warmup.npy`, `metrics_lstm_test.json`



In [13]:
ART = "artifacts"

def build_lstm_ae(seq_len, n_feats, Tconf):
    In = layers.Input(shape=(seq_len, n_feats))
    x = layers.LSTM(Tconf["enc1"], return_sequences=True)(In); x = layers.Dropout(Tconf.get("dropout",0.2))(x)
    x = layers.LSTM(Tconf["enc2"], return_sequences=True)(x);  x = layers.Dropout(Tconf.get("dropout",0.2))(x)
    z = layers.LSTM(Tconf["latent"])(x);                        z = layers.Dropout(Tconf.get("dropout",0.2))(z)
    y = layers.RepeatVector(seq_len)(z)
    y = layers.LSTM(Tconf["enc2"], return_sequences=True)(y);   y = layers.Dropout(Tconf.get("dropout",0.2))(y)
    y = layers.LSTM(Tconf["enc1"], return_sequences=True)(y);   y = layers.Dropout(Tconf.get("dropout",0.2))(y)
    Out = layers.TimeDistributed(layers.Dense(n_feats))(y)
    return models.Model(In, Out, name="lstm_ae")

with open("config.yaml","r",encoding="utf-8") as f:
    CFG = yaml.safe_load(f)
T = CFG["training_lstm"]

# load TEST arrays
Xte = np.load(os.path.join(ART,"X_test.npy"))
yte = np.load(os.path.join(ART,"y_test.npy")).astype(int)
L, D = Xte.shape[1], Xte.shape[2]
print(f"[TEST] Xte={Xte.shape} positives={int(yte.sum())}")

# rebuild & load weights
model_te = build_lstm_ae(L, D, T)
model_te.load_weights(os.path.join(ART,"lstm_ae_best.weights.h5"))

# load VAL variance basis
feat_var = np.load(os.path.join(ART,"feat_var_lstm.npy")).astype(np.float32)
denom = np.clip(feat_var, 1e-8, None)[None,None,:]

# compute raw scores
def score_vnorm_mean_mse(model, X, denom, bs=128):
    scores = np.zeros(len(X), dtype=np.float32)
    for i in range(0,len(X),bs):
        xb = X[i:i+bs].astype(np.float32, copy=False)
        xh = model.predict_on_batch(xb)
        resid2 = (xb - xh)**2 / denom
        scores[i:i+bs] = resid2.mean(axis=(1,2))
    return scores

s_te_raw = score_vnorm_mean_mse(model_te, Xte, denom, bs=128)
np.save(os.path.join(ART,"scores_lstm_test.npy"), s_te_raw)

pr_auc_raw = float(average_precision_score(yte, s_te_raw))
prec_raw, rec_raw, thr_raw = precision_recall_curve(yte, s_te_raw)
pr_curve_raw = np.c_[rec_raw[:-1], prec_raw[:-1], thr_raw]
np.save(os.path.join(ART,"pr_lstm_test.npy"), pr_curve_raw)

print(f"[TEST] PR-AUC raw(varnorm)={pr_auc_raw:.3f}")


[TEST] Xte=(5660, 80, 15) positives=407
[TEST] PR-AUC raw(varnorm)=0.415


In [16]:
ART = "artifacts"
with open("config.yaml","r",encoding="utf-8") as f:
    CFG = yaml.safe_load(f)
E = CFG["evaluation"]


warm_cfg = E.get("calibration",{}).get("warmup",{})
WARM_EN = bool(warm_cfg.get("enable",True))
K_WARM = int(warm_cfg.get("K",50))


s_te_raw = np.load(os.path.join(ART,"scores_lstm_test.npy"))
yte      = np.load(os.path.join(ART,"y_test.npy")).astype(int)
ids_val_path, ids_test_path = os.path.join(ART,"unit_ids_val.npy"), os.path.join(ART,"unit_ids_test.npy")
s_val_path = os.path.join(ART,"scores_lstm_val.npy")
have_ids = all(os.path.exists(p) for p in [ids_val_path, ids_test_path, s_val_path])

def warmup_zscore(scores, unit_ids, K=50):
    scores = scores.astype(np.float32, copy=True)
    out = scores.copy()
    for uid in np.unique(unit_ids):
        idx = np.where(unit_ids==uid)[0]
        base = scores[idx[:min(K,len(idx))]]
        mu, sd = float(base.mean()), float(base.std())
        if sd < 1e-8: sd=1.0
        out[idx] = (scores[idx]-mu)/sd
    return out

if WARM_EN and have_ids:
    ids_val = np.load(ids_val_path); ids_test = np.load(ids_test_path)
    s_val_raw = np.load(s_val_path)
    s_val_adj = warmup_zscore(s_val_raw, ids_val, K=K_WARM)
    s_te_adj  = warmup_zscore(s_te_raw,  ids_test, K=K_WARM)

    pr_auc_warm = float(average_precision_score(yte, s_te_adj))
    print(f"[TEST] PR-AUC warmup(varnorm)={pr_auc_warm:.3f} (K={K_WARM})")

    
    yva = np.load(os.path.join(ART,"y_val.npy")).astype(int)
    pmin,pmax,step = E["sweep"]["pmin"],E["sweep"]["pmax"],E["sweep"]["step"]
    best = {"f1":-1}
    for p in np.arange(pmin,pmax+1e-9,step):
        thr = np.percentile(s_val_adj,p)
        yhat = (s_val_adj>=thr).astype(int)
        f1 = f1_score(yva,yhat,zero_division=0)
        if f1>best["f1"]: best={"p":float(p),"thr":float(thr),"f1":float(f1)}

   
    yhat_te = (s_te_adj>=best["thr"]).astype(int)
    prec,rec,f1,_ = precision_recall_fscore_support(yte,yhat_te,average="binary",zero_division=0)

    
    np.save(os.path.join(ART,"scores_lstm_test_varnorm_warmup.npy"), s_te_adj)
    prec_w, rec_w, thr_w = precision_recall_curve(yte,s_te_adj)
    pr_curve_warm = np.c_[rec_w[:-1], prec_w[:-1], thr_w]
    np.save(os.path.join(ART,"pr_lstm_test_warmup.npy"), pr_curve_warm)

    
    val_payload = json.load(open(os.path.join(ART,"metrics_lstm_val.json")))
    metrics_test = {
        "pr_auc_raw": float(average_precision_score(yte,s_te_raw)),
        "pr_auc_warmup": pr_auc_warm,
        "calibration": {"warmup":{"enabled":True,"K":K_WARM}},
        "val_percentile": best["p"], "threshold_value": best["thr"],
        "confusion_at_chosen":{
            "precision": float(prec),"recall":float(rec),"f1":float(f1),
            "tp": int(((yhat_te==1)&(yte==1)).sum()),
            "fp": int(((yhat_te==1)&(yte==0)).sum()),
            "fn": int(((yhat_te==0)&(yte==1)).sum()),
            "tn": int(((yhat_te==0)&(yte==0)).sum()),
            "n_windows": int(len(yte)),"n_pos":int(yte.sum())
        }
    }
    with open(os.path.join(ART,"metrics_lstm_test.json"),"w") as f: json.dump(metrics_test,f,indent=2)
    print(f"[TEST|varnorm+warmup] F1={f1:.3f} P={prec:.3f} R={rec:.3f} PR-AUC={pr_auc_warm:.3f} pct≈{best['p']:.2f}")
else:
    print("[INFO] Warm-up disabled or missing ids — only raw metrics kept.")


[TEST] PR-AUC warmup(varnorm)=0.537 (K=50)
[TEST|varnorm+warmup] F1=0.436 P=0.361 R=0.550 PR-AUC=0.537 pct≈70.20


## 7) Run Summary, Save PR curves & Environment Snapshot
 - Save compact run summary under `runs/`
 - Save library versions to `artifacts/env_snapshot.json` for reproducibility
 - Validation PR, Test PR (raw), Test PR (warm-up)
- (Figures can always be regenerated from the saved `.npy` curve points)


In [19]:
ART = Path("artifacts"); RUNS = Path("runs"); RUNS.mkdir(exist_ok=True)

def safe_read_json(p, default=None):
    try:
        return json.load(open(p, "r", encoding="utf-8"))
    except Exception:
        return default if default is not None else {}

yaml_hash = (ART/"yaml_hash.txt").read_text().strip() if (ART/"yaml_hash.txt").exists() else "unknown"
meta       = safe_read_json(ART/"preprocessing_meta.json", {})
val_payload= safe_read_json(ART/"metrics_lstm_val.json", {})
test_payload= safe_read_json(ART/"metrics_lstm_test.json", {})

pr_auc_raw  = float(test_payload.get("pr_auc_raw", test_payload.get("pr_auc", float("nan"))))
pr_auc_warm = test_payload.get("pr_auc_warmup", None)
use_warmup  = (pr_auc_warm is not None) and (not np.isnan(pr_auc_warm)) and (pr_auc_warm >= pr_auc_raw)

summary = {
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "yaml_hash": yaml_hash,
    "data": {
        "dataset": "FD001",
        "seq_len": meta.get("seq_len"),
        "stride": meta.get("stride"),
        "n_features": meta.get("n_features"),
        "scale": meta.get("scale"),
        "scale_fit_on": meta.get("scale_fit_on"),
        "label_mode": meta.get("label_mode"),
        "label_dilation": meta.get("label_dilation"),
        "anomaly_horizon": meta.get("anomaly_horizon"),
        "dropped_fixed": meta.get("dropped_fixed"),
    },
    "validation": {
        "percentile": val_payload.get("val_percentile"),
        "threshold": val_payload.get("threshold_value"),
        "metrics": val_payload.get("metrics", {}),
    },
    "test": {
        "pr_auc_raw": pr_auc_raw,
        "pr_auc_warmup": None if not use_warmup else float(pr_auc_warm),
        "used_calibration": "warmup_zscore" if use_warmup else "none",
        "metrics_payload": test_payload,
    },
}

rid = time.strftime("%Y-%m-%d_%H%M") + f"_cfg_{str(yaml_hash)[:6]}"
rdir = RUNS / rid
rdir.mkdir(parents=True, exist_ok=True)
with open(rdir/"run_summary.json","w",encoding="utf-8") as f:
    json.dump(summary, f, indent=2)

with open(ART/"LATEST_RUN.txt","w") as f:
    f.write(str(rdir) + "\n")

print(f"VAL PR-AUC:   {summary['validation']['metrics'].get('pr_auc', 'n/a')}")
print(f"TEST PR-AUC (raw):   {pr_auc_raw:.3f}")
print("TEST PR-AUC (warmup):", "n/a" if not use_warmup else f"{pr_auc_warm:.3f}  <-- best")
print("Run folder:", rdir)


VAL PR-AUC:   0.8375025879916973
TEST PR-AUC (raw):   0.415
TEST PR-AUC (warmup): 0.537  <-- best
Run folder: runs\2025-09-21_1456_cfg_2cff7a


In [21]:
ART = "artifacts"

def save_pr(np_file, out_png, title):
    arr = np.load(os.path.join(ART, np_file))
    # arr columns: [recall, precision, threshold]
    recall, precision = arr[:,0], arr[:,1]
    plt.figure()
    plt.plot(recall, precision, linewidth=2)
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(title)
    plt.grid(True, alpha=0.3); plt.tight_layout()
    plt.savefig(os.path.join(ART, out_png), dpi=150); plt.close()

if os.path.exists(os.path.join(ART,"pr_lstm_val.npy")):
    save_pr("pr_lstm_val.npy", "pr_lstm_val.png", "PR Curve — Validation")

if os.path.exists(os.path.join(ART,"pr_lstm_test.npy")):
    save_pr("pr_lstm_test.npy", "pr_lstm_test.png", "PR Curve — Test (raw)")

if os.path.exists(os.path.join(ART,"pr_lstm_test_warmup.npy")):
    save_pr("pr_lstm_test_warmup.npy", "pr_lstm_test_warmup.png", "PR Curve — Test (warm-up)")

print("Saved PR curves (if available) to artifacts/.")

Saved PR curves (if available) to artifacts/.


In [24]:
ART = Path("artifacts")
env = {
    "python": sys.version.split()[0],
    "platform": sys.platform,
    "packages": {}
}

for pkg, imp in {
    "tensorflow": "tensorflow",
    "numpy": "numpy",
    "pandas": "pandas",
    "scikit-learn": "sklearn",
    "PyYAML": "yaml",
    "matplotlib": "matplotlib"
}.items():
    try:
        mod = __import__(imp)
        ver = getattr(mod, "__version__", "unknown")
        env["packages"][pkg] = ver
    except Exception:
        env["packages"][pkg] = "not_found"

with open(ART/"env_snapshot.json","w",encoding="utf-8") as f:
    json.dump(env, f, indent=2)

print("Wrote artifacts/env_snapshot.json")
print(env)


Wrote artifacts/env_snapshot.json
{'python': '3.10.11', 'platform': 'win32', 'packages': {'tensorflow': '2.10.1', 'numpy': '1.23.5', 'pandas': '2.3.2', 'scikit-learn': '1.7.2', 'PyYAML': '6.0.2', 'matplotlib': '3.10.6'}}
